One shot predict, autoregressive = True

In [1]:
from functions import *
import math
import time
from torch.nn.utils import clip_grad_value_

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-17 19:22:19 [__init__.py:239] Automatically detected platform cuda.


In [ ]:
global_config = GlobalConfig(
    model_name="unsloth/gemma-3-4b-pt-unsloth-bnb-4bit",
    # model_name='unsloth/Qwen2.5-Coder-7B-bnb-4bit',
    r=64,
    data_path='/home/zhenlan/Desktop/Projects/ARC2/Data/ARC-AGI-2-main/combined_data.json',
    tokenization="oneshot",
    max_length=8192,
    autoregressive=True,
    epochs = 80,
    NeedPosition=True
)
global_config.save_to_json()

Deleted folder and contents: ../../Model/model_7
Created folder: ../../Model/model_7


In [3]:
if global_config.NeedPosition:
    model, PosEmbedModel = get_gemma_model(model_name=global_config.model_name, \
                            head_dim=global_config.lm_head_dim, \
                            max_seq_length=global_config.max_length,\
                            NeedPosition=global_config.NeedPosition,\
                            isTrain=True)
else:
    model = get_gemma_model(model_name=global_config.model_name, \
                            head_dim=global_config.lm_head_dim, \
                            max_seq_length=global_config.max_length,\
                            NeedPosition=global_config.NeedPosition,\
                            isTrain=True)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Not a vision language model


In [4]:
with open(global_config.data_path, 'r') as f:
    data = json.load(f)

#### Fine-tune embedding

In [5]:
epochs = 10
accumulation_steps = 32
lr = 3e-5
clip = 3e-3
MAX_LEN = 3600

In [6]:
if 'PosEmbedModel' in globals():
    trainable_params = [param for param in model.parameters() if param.requires_grad] + [param for param in globals()['PosEmbedModel'].parameters() if param.requires_grad]
else:
    trainable_params = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params,lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()

In [7]:
start_time = time.time()
separated_loss = global_config.tokenization == "oneshot" and global_config.autoregressive
if separated_loss:
    train_loss1 = 0
    train_loss2 = 0
    val_loss1 = 0
    val_loss2 = 0
else:
    train_loss = 0
    val_loss = 0
for epoch in range(epochs):
    model.train()
    for i,out in enumerate(data_gen(data,True,MAX_LEN,autoregressive=global_config.autoregressive,\
                                    tokenize_func=global_config.tokenizer,IsDecode=False,\
                                    NeedPosition=global_config.NeedPosition)):
        x, y = out["input_tokens"], out["target_tokens"]
        if global_config.NeedPosition:
            PosEmbedModel.reset_parameters(out["row_indices"], out["col_indices"])
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            yhat = model(x).logits
            if separated_loss:
                l = out["len_input"]
                _,L,V = yhat.shape
                loss1 = loss_fn(yhat[0,:l].view(l, V),y[0,:l].view(-1))
                loss2 = loss_fn(yhat[0,l:].view(L-l, V),y[0,l:].view(-1))
                loss = loss1 * 0.6 / math.sqrt(epoch+1) + loss2
            else:
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        if separated_loss:
            train_loss1 += loss1.item()
            train_loss2 += loss2.item()
        else:
            train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
            
    model.eval()
    for j,out in enumerate(data_gen(data,False,MAX_LEN*4,autoregressive=global_config.autoregressive,\
                                    tokenize_func=global_config.tokenizer,IsDecode=False,\
                                    NeedPosition=global_config.NeedPosition)):
        x, y = out["input_tokens"], out["target_tokens"]
        if global_config.NeedPosition:
            PosEmbedModel.reset_parameters(out["row_indices"], out["col_indices"])        
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                yhat = model(x).logits
                if separated_loss:
                    l = out["len_input"]
                    _,L,V = yhat.shape
                    loss1 = loss_fn(yhat[0,:l].view(l, V),y[0,:l].view(-1))
                    loss2 = loss_fn(yhat[0,l:].view(L-l, V),y[0,l:].view(-1))
                else:
                    loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        if separated_loss:
            val_loss1 += loss1.item()
            val_loss2 += loss2.item()
        else:
            val_loss += loss.item()
    if separated_loss:    
        print(f"Epoch {epoch+1} - Train Loss1: {train_loss1/i:.4f} - Train Loss2: {train_loss2/i:.4f} - Val Loss1: {val_loss1/j:.4f} - Val Loss2: {val_loss2/j:.4f}")
        train_loss1 = 0
        train_loss2 = 0
        val_loss1 = 0
        val_loss2 = 0        
    else:
        print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
        train_loss = 0
        val_loss = 0
    

end_time = time.time()
total_time = end_time - start_time
print(f"Total training time: {total_time/60:.2f} minutes")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!
Epoch 1 - Train Loss1: 4.7379 - Train Loss2: 4.7863 - Val Loss1: 3.5062 - Val Loss2: 3.4761
Epoch 2 - Train Loss1: 3.1548 - Train Loss2: 3.1108 - Val Loss1: 2.8763 - Val Loss2: 2.8419
Epoch 3 - Train Loss1: 2.7719 - Train Loss2: 2.7039 - Val Loss1: 2.6854 - Val Loss2: 2.6728
Epoch 4 - Train Loss1: 2.6561 - Train Loss2: 2.6002 - Val Loss1: 2.6392 - Val Loss2: 2.6187
Epoch 5 - Train Loss1: 2.6126 - Train Loss2: 2.5574 - Val Loss1: 2.6169 - Val Loss2: 2.5751
Epoch 6 - Train Loss1: 2.5804 - Train Loss2: 2.5182 - Val Loss1: 2.6019 - Val Loss2: 2.5800
Epoch 7 - Train Loss1: 2.5707 - Train Loss2: 2.5179 - Val Loss1: 2.5910 - Val Loss2: 2.5726
Epoch 8 - Train Loss1: 2.5551 - Train Loss2: 2.4926 - Val Loss1: 2.5435 - Val Loss2: 2.5276
Epoch 9 - Train Loss1: 2.5455 - Train Loss2: 2.4900 - Val Loss1: 2.5647 - Val Loss2: 2.5334
Epoch 10 - Train Loss1: 2.5425 - Train Loss2: 2.4761 - Val Loss1: 2.5926 - Val Loss2: 2.5584
Total training time: 135.

In [8]:
torch.save(model.lm_head.state_dict(), global_config.folder + 'lm_heads_weights.pth')
if global_config.NeedPosition:
    torch.save(PosEmbedModel.state_dict(), global_config.folder + 'PosEmbedModel.pth')

#### Fine-Tune QLORA

In [ ]:
epochs = global_config.epochs
accumulation_steps = 32
lr = 4e-5
clip = 4e-3
MAX_LEN = 3600
model.max_seq_length = MAX_LEN
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = global_config.r,           # Larger = higher accuracy, but might overfit
    lora_alpha = global_config.r,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)
if 'PosEmbedModel' in globals():
    trainable_params = [param for param in model.parameters() if param.requires_grad] + [param for param in globals()['PosEmbedModel'].parameters() if param.requires_grad]
else:
    trainable_params = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params,lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()

Unsloth: Making `model.base_model.model.model` require gradients


In [15]:
start_time = time.time()
separated_loss = global_config.tokenization == "oneshot" and global_config.autoregressive
if separated_loss:
    train_loss1 = 0
    train_loss2 = 0
    val_loss1 = 0
    val_loss2 = 0
else:
    train_loss = 0
    val_loss = 0
for epoch in range(epochs):
    model.train()
    for i,out in enumerate(data_gen(data,True,MAX_LEN,autoregressive=global_config.autoregressive,\
                                    tokenize_func=global_config.tokenizer,IsDecode=False,\
                                    NeedPosition=global_config.NeedPosition)):
        x, y = out["input_tokens"], out["target_tokens"]
        if global_config.NeedPosition:
            PosEmbedModel.reset_parameters(out["row_indices"], out["col_indices"])
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            yhat = model(x).logits
            if separated_loss:
                l = out["len_input"]
                _,L,V = yhat.shape
                loss1 = loss_fn(yhat[0,:l].view(l, V),y[0,:l].view(-1))
                loss2 = loss_fn(yhat[0,l:].view(L-l, V),y[0,l:].view(-1))
                loss = loss1 * 0.6 / math.sqrt(epoch+1) + loss2
            else:
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        if separated_loss:
            train_loss1 += loss1.item()
            train_loss2 += loss2.item()
        else:
            train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
            
    model.eval()
    for j,out in enumerate(data_gen(data,False,MAX_LEN*4,autoregressive=global_config.autoregressive,\
                                    tokenize_func=global_config.tokenizer,IsDecode=False,\
                                    NeedPosition=global_config.NeedPosition)):
        x, y = out["input_tokens"], out["target_tokens"]
        if global_config.NeedPosition:
            PosEmbedModel.reset_parameters(out["row_indices"], out["col_indices"])        
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                yhat = model(x).logits
                if separated_loss:
                    l = out["len_input"]
                    _,L,V = yhat.shape
                    loss1 = loss_fn(yhat[0,:l].view(l, V),y[0,:l].view(-1))
                    loss2 = loss_fn(yhat[0,l:].view(L-l, V),y[0,l:].view(-1))
                else:
                    loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        if separated_loss:
            val_loss1 += loss1.item()
            val_loss2 += loss2.item()
        else:
            val_loss += loss.item()
    if separated_loss:    
        print(f"Epoch {epoch+1} - Train Loss1: {train_loss1/i:.4f} - Train Loss2: {train_loss2/i:.4f} - Val Loss1: {val_loss1/j:.4f} - Val Loss2: {val_loss2/j:.4f}")
        train_loss1 = 0
        train_loss2 = 0
        val_loss1 = 0
        val_loss2 = 0        
    else:
        print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
        train_loss = 0
        val_loss = 0
    

end_time = time.time()
total_time = end_time - start_time
print(f"Total training time: {total_time/60:.2f} minutes")

Epoch 1 - Train Loss1: 2.0849 - Train Loss2: 2.0513 - Val Loss1: 2.0733 - Val Loss2: 2.1406
Epoch 2 - Train Loss1: 2.1047 - Train Loss2: 2.0628 - Val Loss1: 2.2381 - Val Loss2: 2.3345
Epoch 3 - Train Loss1: 2.0372 - Train Loss2: 2.0100 - Val Loss1: 2.2736 - Val Loss2: 2.3385
Epoch 4 - Train Loss1: 2.0495 - Train Loss2: 2.0077 - Val Loss1: 2.1048 - Val Loss2: 2.1257
Epoch 5 - Train Loss1: 2.0450 - Train Loss2: 1.9978 - Val Loss1: 2.1943 - Val Loss2: 2.2697
Epoch 6 - Train Loss1: 2.0305 - Train Loss2: 1.9893 - Val Loss1: 2.1021 - Val Loss2: 2.2068
Epoch 7 - Train Loss1: 2.0236 - Train Loss2: 1.9737 - Val Loss1: 2.0183 - Val Loss2: 2.1501
Epoch 8 - Train Loss1: 2.0072 - Train Loss2: 1.9631 - Val Loss1: 2.1606 - Val Loss2: 2.2544


KeyboardInterrupt: 

In [ ]:
model.save_pretrained(global_config.folder + 'finetuned_model')
torch.save(model.lm_head.state_dict(), global_config.folder + 'lm_heads_weights.pth')
if global_config.NeedPosition:
    torch.save(PosEmbedModel.state_dict(), global_config.folder + 'PosEmbedModel.pth')

/home/zhenlan/anaconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


#### Decoding

In [ ]:
# from functions import *
# model = get_gemma_model(model_name="unsloth/gemma-3-4b-pt-unsloth-bnb-4bit",
#                          head_dim=48,
#                          lm_head_path='../../Model/lm_heads_weights_pt_oneshot.pth',
#                          peft_path='../../Model/merged_model_pt_oneshot',
#                          isTrain=False)

In [ ]:
# import json
# output_path = '/home/zhenlan/Desktop/Projects/ARC2/Data/ARC-AGI-2-main/combined_data.json'
# with open(output_path, 'r') as f:
#     data = json.load(f)
# MAX_LEN = 3600

In [ ]:
# val_loss1 = 0
# val_loss2 = 0
# loss_fn = torch.nn.CrossEntropyLoss()
# model.eval()
# for j,(x,y,l) in enumerate(data_gen(data,True,MAX_LEN*4,autoregressive=True,tokenize_func=tokenize_oneshot,IsDecode=False)):
#     with torch.no_grad():
#         with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
#             yhat = model(x).logits
#             _,L,V = yhat.shape
#             loss1 = loss_fn(yhat[0,:l].view(l, V),y[0,:l].view(-1))
#             loss2 = loss_fn(yhat[0,l:].view(L-l, V),y[0,l:].view(-1))
#     val_loss1 += loss1.item()
#     val_loss2 += loss2.item()
# print(f"Val Loss1: {val_loss1/j:.4f} - Val Loss2: {val_loss2/j:.4f}")

In [ ]:
train_correct = 0
train_yhat_none = 0
train_yhat_wrong_shape = 0
train_yhat_mismatch = 0

val_correct = 0
val_yhat_none = 0
val_yhat_wrong_shape = 0
val_yhat_mismatch = 0

model.eval()
decoder = global_config.decoder(model)

# Training data
for i, out in enumerate(data_gen(data, True, MAX_LEN*4, autoregressive=global_config.autoregressive, tokenize_func=global_config.tokenizer, IsDecode=True)):
    yhat = decoder.decode(out[0])
    y = out[1]
    
    if yhat is None:
        train_yhat_none += 1
    elif yhat.shape != y.shape:
        train_yhat_wrong_shape += 1
    elif not np.all(y == yhat):
        train_yhat_mismatch += 1
    else:
        train_correct += 1

# Validation data
for j, out in enumerate(data_gen(data, False, MAX_LEN*4, autoregressive=global_config.autoregressive, tokenize_func=global_config.tokenizer, IsDecode=True)):
    yhat = decoder.decode(out[0])
    y = out[1]
    
    if yhat is None:
        val_yhat_none += 1
    elif yhat.shape != y.shape:
        val_yhat_wrong_shape += 1
    elif not np.all(y == yhat):
        val_yhat_mismatch += 1
    else:
        val_correct += 1

# Calculate total samples
train_total = i + 1
val_total = j + 1

# Calculate percentages
train_none_pct = (train_yhat_none / train_total) * 100
train_wrong_shape_pct = (train_yhat_wrong_shape / train_total) * 100
train_mismatch_pct = (train_yhat_mismatch / train_total) * 100
train_correct_pct = (train_correct / train_total) * 100

val_none_pct = (val_yhat_none / val_total) * 100
val_wrong_shape_pct = (val_yhat_wrong_shape / val_total) * 100
val_mismatch_pct = (val_yhat_mismatch / val_total) * 100
val_correct_pct = (val_correct / val_total) * 100

# Print results
print("Training Error Breakdown:")
print(f"  yhat is None: {train_none_pct:.2f}%")
print(f"  yhat wrong shape: {train_wrong_shape_pct:.2f}%")
print(f"  yhat mismatch: {train_mismatch_pct:.2f}%")
print(f"  Correct: {train_correct_pct:.2f}%")

print("\nValidation Error Breakdown:")
print(f"  yhat is None: {val_none_pct:.2f}%")
print(f"  yhat wrong shape: {val_wrong_shape_pct:.2f}%")
print(f"  yhat mismatch: {val_mismatch_pct:.2f}%")
print(f"  Correct: {val_correct_pct:.2f}%")